In [3]:
import geopandas as gpd
from shapely.geometry import LineString

# Load the original line layer with 'pct' attribute
gdf = gpd.read_file("czech_linie.gpkg")

# Function to trim line according to the percentage from origin to destination
def trim_line_by_pct(row):
    line = row.geometry
    pct = row.pct

    if not isinstance(line, LineString) or pct is None:
        return line

    if pct >= 100:
        return line

    target_length = line.length * (pct / 100)
    trimmed_coords = []
    current_length = 0

    for i in range(len(line.coords) - 1):
        start = line.coords[i]
        end = line.coords[i + 1]
        segment = LineString([start, end])
        seg_length = segment.length

        if current_length + seg_length < target_length:
            trimmed_coords.append(start)
            current_length += seg_length
        else:
            remaining = target_length - current_length
            if remaining > 0:
                point = segment.interpolate(remaining)
                trimmed_coords.append(start)
                trimmed_coords.append((point.x, point.y))
            break

    if len(trimmed_coords) < 2:
        return LineString([line.coords[0], line.interpolate(target_length)])

    return LineString(trimmed_coords)

# Apply the trimming function
gdf["geometry"] = gdf.apply(trim_line_by_pct, axis=1)

# Export the result to a new GeoPackage
output_path = "czech_linie_trimmed.gpkg"
gdf.to_file(output_path, driver="GPKG", layer="trimmed_lines")
